In [1]:
import sklearn


print('The scikit-learn version is {}.'.format(sklearn.__version__))


The scikit-learn version is 1.2.2.


# 1. Imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import os
import numpy as np
from torchvision import datasets, models, transforms

from torchvision.models import resnet18, ResNet18_Weights
import torch.optim as optim


import random
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split


from sklearn.model_selection import GroupKFold
from sklearn.model_selection import StratifiedGroupKFold
from imblearn.over_sampling import RandomOverSampler



from codes.utils import stratified_train_test_group_kfold
from codes.utils import model_test_classification
from codes.utils import accuracy_classification
from codes.utils import mlp_class_cross_val_final_test

from codes.classification_codes import mlp_class_gridsearch


torch.backends.cudnn.deterministic = True
random.seed(1)
torch.manual_seed(1)
torch.cuda.manual_seed(1)
np.random.seed(1)

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

# 2. Data

In [2]:
with open("Data/MoS2_Analysis_Data_trained2", "rb") as fp:   # Unpickling
    MoS2_Proj1_Class_Data = pickle.load(fp)

#features = MoS2_ImageNet_100_data[0]
df = pd.DataFrame(MoS2_Proj1_Class_Data)
print(df.keys())
T_dict= {900:0, 950:1, 1000:2}
T_target = np.array([T_dict[item] for item in df['T']])

features = np.array(list(df['ImageNet']))
sampleId = np.array(list(df['sampleId']))


X = features
Y = np.array(T_target)
groups = np.array(sampleId)

train_val_groups, train_val_X, train_val_Y, test_X, test_Y = stratified_train_test_group_kfold(X, Y, groups, n_splits=10, test_fold=0)


#Y = Y.flatten()

oversample = RandomOverSampler(sampling_strategy='not majority')

print(train_val_X.shape)
print(train_val_Y.shape)
print(test_X.shape)
print(test_Y.shape)






Index(['sampleId', 'sampleLabel', 'image', 'T', 'ImageNet', 'MicroNet'], dtype='object')
<class 'generator'>
(235, 100)
(235,)
(27, 100)
(27,)


# 3. Runing Cross-Validation

In [3]:
#!mkdir -p 'Models/classification/ImageNet/aug3/MLP'

In [4]:
# the paramters below were optimized
#lr: 0.01, batch size: 16, dropout: 0.35, epochs: 32, l1: 600, l2: 50
                        
                        
Learning_rate = [0.01]
Drop_out = [0.35]#[0.0, 0.2, 0.35, 0.5]
Batch_size = [16]#[16, 32, 100]
L1 = [600]#[600, 400, 300, 200]
L2 = [50]# [150, 100, 50]

def cross_10_folds_mlp(train_val_X, train_val_Y):

    best_train = []
    best_val = []
    best_variables = []
    best_performance_record = []
    for fold in range(10):
        
        model_path = f'classification/ImageNet/aug3/MLP/{fold}_model.pth'
        group, train_X, train_Y, val_X, val_Y = stratified_train_test_group_kfold(train_val_X, train_val_Y, train_val_groups, n_splits=10, test_fold=fold)
        train_X, train_Y = oversample.fit_resample(train_X, train_Y)
        
        #train_X, val_X, train_Y, val_Y = train_test_split(train_val_X, train_val_Y, test_size=0.1,stratify=train_val_Y,random_state=fold+30)#42
        best_train_acc, best_val_acc, best_record, hyper=mlp_class_gridsearch(train_X, train_Y, val_X, val_Y,Learning_rate, L1, L2, Drop_out,Batch_size, fold,model_path)
        #best_train_acc, best_val_acc, trainf1, valf1, best_perf_record, hyper=cnn_class_train_fn(train_X, train_Y, val_X, val_Y, Learning_rate, Batch_size, fold)

        best_train.append(best_train_acc)
        best_val.append(best_val_acc)
        best_variables.append(hyper)
        best_performance_record.append(best_record)

        print(f'fold: {fold} done!')
    return best_train, best_val, best_variables, best_performance_record


best_train, best_val, best_variables, best_performance_record = cross_10_folds_mlp(train_val_X, train_val_Y)



<class 'generator'>
Early stopped training at epoch 14
Accuracy of the network on the 315 train images: 93.0 %
Accuracy of the network on the 24 val images: 79.2 %
fold: 0 done!
<class 'generator'>
Early stopped training at epoch 25
Accuracy of the network on the 318 train images: 97.8 %
Accuracy of the network on the 24 val images: 70.8 %
fold: 1 done!
<class 'generator'>
Early stopped training at epoch 14
Accuracy of the network on the 318 train images: 95.0 %
Accuracy of the network on the 24 val images: 66.7 %
fold: 2 done!
<class 'generator'>
Early stopped training at epoch 19
Accuracy of the network on the 315 train images: 90.2 %
Accuracy of the network on the 24 val images: 79.2 %
fold: 3 done!
<class 'generator'>
Early stopped training at epoch 14
Accuracy of the network on the 315 train images: 95.2 %
Accuracy of the network on the 24 val images: 70.8 %
fold: 4 done!
<class 'generator'>
Early stopped training at epoch 28
Accuracy of the network on the 315 train images: 97.1 %

# 4. Test Models

In [5]:
class MLP(nn.Module):
    def __init__(self, l1=120, l2=84, p = 0.2):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(100, l1)
        self.fc2 = nn.Linear(l1, l2)
        self.fc3 = nn.Linear(l2, 3)
        #self.activ = torch.nn.Sigmoid()
        self.dropout = nn.Dropout(p)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

l1, l2, p = 600, 50, 0.35
trained_model = model = MLP(l1, l2, p).to(device)

In [6]:

#trained_model = pretrained_model()
root_path = f'classification/ImageNet/aug3/MLP'

best_test, confusion_matrix_test = mlp_class_cross_val_final_test(trained_model, test_X, test_Y, 'test', root_path)


print(best_test)
print(f'test_acc_mean: {np.mean(best_test) :.3f}, std: {np.std(best_test) :.3f}')
print("......")

def cm_to_plot(best_test):

    max_test = np.max(best_test)
    idx_cm = []
    for index, item in enumerate(best_test):
        if item == max_test:
            idx_cm.append(index)

    print('idx_cm: ', idx_cm)
    print('max_test_acc: ', max_test)

    print(confusion_matrix_test[idx_cm[0]])
cm_to_plot(best_test)
print('5th confusion matrix: ', [list(item) for item in confusion_matrix_test[4]])


cm_list = []
for cm in confusion_matrix_test:
    cm_list.append([list(item) for item in cm])
#print(cm_list)

print(len(cm_list))
print('avg cm: ', [list(item) for item in np.mean(cm_list, axis=0)])


Accuracy of the network on the 27 test images: 66.7 %
Accuracy of the network on the 27 test images: 66.7 %
Accuracy of the network on the 27 test images: 66.7 %
Accuracy of the network on the 27 test images: 66.7 %
Accuracy of the network on the 27 test images: 70.4 %
Accuracy of the network on the 27 test images: 81.5 %
Accuracy of the network on the 27 test images: 77.8 %
Accuracy of the network on the 27 test images: 74.1 %
Accuracy of the network on the 27 test images: 70.4 %
Accuracy of the network on the 27 test images: 66.7 %
[66.66666666666667, 66.66666666666667, 66.66666666666667, 66.66666666666667, 70.37037037037037, 81.48148148148148, 77.77777777777777, 74.07407407407408, 70.37037037037037, 66.66666666666667]
test_acc_mean: 70.741, std: 5.092
......
idx_cm:  [5]
max_test_acc:  81.48148148148148
[[ 1  2  0]
 [ 0 12  1]
 [ 0  2  9]]
5th confusion matrix:  [[1, 2, 0], [2, 10, 1], [0, 3, 8]]
10
avg cm:  [[1.0, 2.0, 0.0], [1.2, 10.2, 1.6], [0.0, 3.1, 7.9]]
